# Experimentación métodos iterativos

In [2]:
import os
import numpy as np
import subprocess as sp

In [3]:
dir = "../data"

## Funciones auxiliares

In [4]:
def generar_sistema_edd(tam, sol, tipo):
    A = np.random.randint(1, 40, (tam, tam))
    suma = np.sum(np.abs(A), axis=1)
    A += np.diag(suma)

    b = A @ sol

    if not os.path.exists(f"{dir}/{tipo}"):
        os.makedirs(f"{dir}/{tipo}")

    np.savetxt(f"{dir}/{tipo}/matriz_edd_{tam}.txt", np.hstack((A, b[:, np.newaxis])), "%d", " ", comments="", header=f"{tam}")

def generar_sistema_simetrico(tam, sol, tipo):
    A = np.random.randint(1, 40, (tam, tam))
    A = A.T @ A

    b = A @ sol

    if not os.path.exists(f"{dir}/{tipo}"):
        os.makedirs(f"{dir}/{tipo}")

    np.savetxt(f"{dir}/{tipo}/matriz_sim_{tam}.txt", np.hstack((A, b[:, np.newaxis])), "%d", " ", comments="", header=f"{tam}")

def generar_sistema_triangular(tam, sol, tipo):
    A = np.tril(np.random.randint(1, 40, (tam, tam)))
    b = A @ sol

    if not os.path.exists(f"{dir}/{tipo}"):
        os.makedirs(f"{dir}/{tipo}")

    np.savetxt(f"{dir}/{tipo}/matriz_triang_{tam}.txt", np.hstack((A, b[:, np.newaxis])), "%d", " ", comments="", header=f"{tam}")

In [5]:
def correr_algoritmo(dataset, tipo, tam, metodo, iteraciones, tol):        
    proceso = sp.run(["../src/iterativo", f"{dir}/{dataset}/matriz_{tipo}_{tam}.txt", metodo, str(iteraciones), str(tol)], capture_output=True, text=True)
    proceso.check_returncode()

    return np.array(proceso.stdout.split(" "), dtype=np.float64)


In [6]:
def crear_archivos(n, low, high, tipo):
    for i in range(2, n + 1):
        tam = 2 ** i
        x = np.random.randint(low, high, tam)
        
        generar_sistema_edd(tam, x, tipo)
        generar_sistema_simetrico(tam, x, tipo)
        generar_sistema_triangular(tam, x, tipo)

        np.savetxt(f"{dir}/{tipo}/vector_x_{tam}.txt", x, "%d", " ")

## Generar instancias

In [7]:
n = 8

### Vector solucion chico

In [8]:
crear_archivos(n, 0, 10, "chico")

### Vector solucion mediano

In [9]:
crear_archivos(n, 50, 100, "mediano")

### Vector solucion grande

In [10]:
crear_archivos(n, 200, 300, "grande")

## Correr algoritmos

In [11]:
# res = []
# correr_algoritmo("chico", "LU", 1000, 1e-20)

-Error de aproximacion entre valor actual y final (por cantidad de iteraciones) para matrices de diferentes tamaños
-Tiempo final de computo por iteraciones y por tamaño de matrices
-LU tiempo de computo por tamaño de matriz. Error numerico (comparar con x real).
-Comparar LU con todos los metodos iterativos.

## Generación de Gráficos

### Error

In [12]:
def cargar_soluciones(tipo):
    for i in range(2, n + 1):
        tam = 2 ** i
        xs.append(np.genfromtxt(f"{dir}/{tipo}/vector_x_{tam}.txt"))

In [29]:
skipped = 0
datasets = ["chico", "mediano", "grande"]
algoritmos = ["J", "JS", "GS", "GSS"]
tipos = ["edd", "sim", "triang"]
max_iter = 601
step = 200
tolerancia = 1e-10
resultados = {}

for ds in datasets:
    resultados[f"{ds}"] = {}
    for algoritmo in algoritmos:
        resultados[f"{algoritmo}"] = {}
        for tipo in tipos:
            resultados[f"{ds}"][f"{algoritmo}"][f"{tipo}"] = {}
            for i in range(2, n + 1): # Inicializo arrays
                resultados[f"{ds}"][f"{algoritmo}"][f"{tipo}"][f"T_{2**i}"] = []
            for i in range(2, n + 1):
                for j in range(1, max_iter + 1, step):
                    t = %timeit -r 1 -n 1 -o correr_algoritmo(ds, tipo, 2 ** i, algoritmo, j, tolerancia)
                    res = correr_algoritmo(ds, tipo, 2 ** i, algoritmo, j, tolerancia)
                    resultados[f"{algoritmo}"][f"{tipo}"][f"T_{2**i}"].append(
                        {
                         "niter": j,
                         "tiempo": t,
                         "res": res
                        })


for ds in datasets:
    resultados[f"{ds}"]["LU"] = {}
    for tipo in tipos:
        resultados["LU"][f"{tipo}"] = {}
        for i in range (2, n + 1):
            resultados["LU"][f"{tipo}"][f"T_{2**i}"] = []
        for i in range (2, n + 1):
            t = %timeit -r 5 -n 3 -o correr_algoritmo(ds, tipo, 2 ** i, "LU", 0, tolerancia)
            res = correr_algoritmo(ds, tipo, 2 ** i, "LU", 0, tolerancia)
            resultados["LU"][f"{tipo}"][f"T_{2**i}"].append(
                {
                 "tiempo": t,
                 "res": res
                })
resultados

2.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.52 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.98 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
4.17 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
13.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
55 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
331 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.91 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.01 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.03 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
3.09 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
12.6 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
69.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
331 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.64 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.19 ms ± 0 ns per loop (mean 

{'J': {'edd': {'T_4': [{'niter': 1,
     'tiempo': <TimeitResult : 1.57 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>,
     'res': array([400.666, 405.436, 397.648, 441.572])}],
   'T_8': [{'niter': 1,
     'tiempo': <TimeitResult : 1.63 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>,
     'res': array([433.946, 479.459, 412.952, 380.381, 415.831, 452.621, 465.638,
            446.905])}],
   'T_16': [{'niter': 1,
     'tiempo': <TimeitResult : 2.01 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>,
     'res': array([499.239, 476.727, 472.299, 431.577, 497.608, 472.948, 456.956,
            434.687, 496.636, 499.326, 439.221, 423.399, 457.647, 443.027,
            414.547, 515.657])}],
   'T_32': [{'niter': 1,
     'tiempo': <TimeitResult : 2.78 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>,
     'res': array([532.141, 445.44 , 451.284, 550.016, 472.778, 517.837, 477.796,
            531.782, 509.391, 506.165, 497.537, 513.177, 45

In [36]:
xs = []
print(resultados["LU"]["edd"]["T_256"][-1]["res"] - np.genfromtxt(f"{dir}/grande/vector_x_256.txt"))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
